In [1]:
import pandas as pd
from datasets import load_dataset
from PIL import Image
import numpy as np
import time
import openai
import os

In [ ]:

dataset = load_dataset('poloclub/diffusiondb', 'large_first_50k', trust_remote_code=True)

part-000008.zip:   6%|5         | 31.5M/530M [00:00<?, ?B/s]

part-000009.zip:   0%|          | 0.00/519M [00:00<?, ?B/s]

part-000010.zip:   0%|          | 0.00/459M [00:00<?, ?B/s]

In [ ]:
df = pd.DataFrame(dataset)
df.head()

In [ ]:
df = df['train'].apply(pd.Series)
df.head()

In [ ]:
image = df.loc[0, 'image']
image.show()
image_array = np.array(image)
print(image_array.shape)

In [ ]:
image_rgb = image.convert('RGB')
image_array = np.expand_dims(image_rgb, axis=0)


In [ ]:
print(image_array)

In [ ]:
openai.api_key = os.getenv("OPENAI_API_KEY")

if openai.api_key is None:
    print(' ValueError("API key is missing. Please set the OPENAI_API_KEY environment variable.")')
    openai.api_key=input("whats your API key?")
# Initialize the OpenAI client
client = openai

In [ ]:
def classify_word(word):
    # Define a system message and user prompt to classify each word
    prompt = f"Classify the word '{word}' into one of the following categories: 'illegal', 'explicit', 'violent', 'innocent'. Provide just the category as the output."

    # Call the OpenAI API with the prompt
    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": "You are a helpful assistant that classifies words."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=10
        )
        
        # Parse the response and extract the classification
        classification = response.choices[0].message.content.strip()
        if 'illegal' in classification:
            classification = 'illegal'
        elif 'explicit' in classification:
            classification = 'explicit'
        elif 'violent' in classification:
            classification = 'violent'
        elif 'innocent' in classification:
            classification = 'innocent'
        # Map the response to a unique value
        category_mapping = {
            "illegal": 1,
            "explicit": 2,
            "violent": 3,
            "innocent": 0
        }
        
        # Return the mapped value
        return category_mapping.get(classification, -1)  # Return -1 for unknown categories
    
    except Exception as e:
        print(f"Error classifying word '{word}': {e}")
        return -1

In [ ]:
def numberify(word):
    # Define a system message and user prompt to classify each word
    prompt = f"""Assign the word a number based on the following criteria. The number will be 8 digits. the first 4 are to determine how much censorship the word may be. 
    Ensure a pattern is involved. Filler words will be 1111xxxx. Subjects(non explicit) will be 2222xxxx. Verbs (non explicit) will be 3333xxxx. Then have forward scaling for explicit words. 
    For example the word 'sex' will be 8888xxxx and the word 'rape' will be 9999xxxx. Have intermediate words follow a similar scale. 
    The final four numbers will be determined with the following. The first 2 numbers of the final 4 numbers will be based on word length. The following 2 numbers will be a fine scale of possible explicit value. 
    For example, the word sex will be classified as 88880395. If a word could have potential implications to violence, illegality, drugs, or explicit content, then give it a higher number than 400000. Use a good scale though. words like assassin or shot shouldnt be classified as 9999xxxx. Try to keep a balance with 5555xxxx, 6666xxxx, 7777xxxx, 8888xxxx.
    Respond with just the number! HERE IS THE WORD: {word}"""

    # Call the OpenAI API with the prompt
    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": "You are a helpful assistant that classifies words."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=10
        )
        
        # Parse the response and extract the classification
        classification = response.choices[0].message.content.strip()        
        worded = word
        return classification, worded
    
    except Exception as e:
        print(f"Error classifying word '{word}': {e}")
        return -1

In [ ]:
df.info()

In [ ]:
import re

# Example DataFrame (replace this with your actual DataFrame)

word_classifications = {}
rows = []
words_set = set()
# Loop through the DataFrame and classify words across all rows
for idx, row in df.iterrows():
    prompt = row['prompt']
    for word in re.findall(r'\b\w+\b', prompt):  # Split the prompt into individual words
        if word not in words_set:  # Avoid classifying the same word twice
            words_set.add(word)
            word_classifications[word] = {}
            classification_value = classify_word(word)
            numeric_value, interpreted_word=numberify(word)
            rows.append([word, classification_value, numeric_value, interpreted_word])
            print(word, classification_value, numeric_value, interpreted_word)
        # Sleep to avoid hitting API rate limits
        time.sleep(.5)  # Adjust the sleep time as necessary

# Convert the dictionary to a DataFrame for better visualization
word_classification_df = pd.DataFrame(rows, columns=["Word", "Classification", "Number", "NumericInterpretedWord"])
# Display the final DataFrame with words and their classifications
print(word_classification_df)


In [16]:
print(word_classification_df['Classification'].unique())

[0 3 2 1]
